In [ ]:
import matplotlib.pylab as plt
import seaborn as sns
import numpy as np
import pandas as pd

from experiments.experiment_main import prediction_plot, error_plots_flexible, collect_data, L2, H1, models_color_dict
from src.experiments.Proceedings.parameters import epsilons2try
from experiments.Proceedings.parameters import k, alpha

import ipywidgets as widgets

# Visualizing experiments

Explore how each method behaves for different $\varepsilon$, K, float_precision, sampling method.

In [ ]:
data_experiment_name = "Proceedings"

### Time and accuracy

In [ ]:
from experiments.Proceedings.n_train import n_train
from experiments.Proceedings.n_train import float_precision
from experiments.Proceedings.n_train import sampler

def jupyter_plot_computation_time_accuracy(epsilon, error, float_precision, sampler):
    df, predictions, true_solutions = collect_data(data_experiment_name, k=k, alpha=alpha, n_train=n_train, sampler=[sampler], float_precision=[float_precision])
    filter_dict = {"epsilon": epsilon}
    
    fig, ax = plt.subplots(ncols=2, figsize=(14, 6))
    
    sns.boxenplot(data=df[(df[list(filter_dict)] == pd.Series(filter_dict)).all(axis=1)], y="time", x="model_name",
                  hue="n_train", ax=ax[0])
    ax[0].set_yscale("log")
    ax[0].set_title("Computing time")
    
    sns.boxenplot(data=df[(df[list(filter_dict)] == pd.Series(filter_dict)).all(axis=1)], y=H1, x="model_name",
                  hue="n_train", ax=ax[1])
    ax[1].set_yscale("log")
    ax[1].set_title("Accuracy by number of train sampling points.")
    
    plt.show()

    
widgets.interact(
    jupyter_plot_computation_time_accuracy,
    epsilon=widgets.Dropdown(options=epsilons2try.tolist(), description=r'$\varepsilon$: '),
    error=widgets.Dropdown(options=[L2, H1], description='Error: '),
    float_precision=widgets.Dropdown(options=float_precision, description='Float precision: '),
    sampler=widgets.Dropdown(options=sampler, description='Sampling method: ')
    )

### Varying K, sampling method or float precision

In [ ]:
experiment = "n_train" # "sampling_method" "float_precision"

#### Exploring error curves

In [ ]:
summary_stattistic_functions_dict = {"min": np.min, "max": np.max, "median": np.median, "mean": np.mean} 

def jupyter_plot_error_curves(statistic, error, n_train, float_precision, sampler):
    aggfunc = summary_stattistic_functions_dict[statistic]
    df, predictions, true_solutions = collect_data(data_experiment_name, k=k, alpha=alpha, n_train=[n_train], sampler=[sampler], float_precision=[float_precision])
    fig, ax = plt.subplots(figsize=(12, 6))
    error_plots_flexible(df, ax, error_name=error, label_var="model_name", x_var="epsilon",
                         aggfunc=aggfunc, color_dict=models_color_dict, n_train=n_train)
    ax.set_ylim((1e-9, 1))
    plt.show()
    
widgets.interact(
    jupyter_plot_error_curves,
    statistic=widgets.Dropdown(options=list(summary_stattistic_functions_dict.keys()), description='Summary statistic: '),
    error=widgets.Dropdown(options=[L2, H1], description='Error: '),
    n_train=widgets.Dropdown(options=n_train, description='N train: '),
    float_precision=widgets.Dropdown(options=float_precision, description='Float precision: '),
    sampler=widgets.Dropdown(options=sampler, description='Sampling method: ')
    )

In [ ]:
if experiment == "n_train":
    from experiments.Proceedings.n_train import n_train
    from experiments.Proceedings.n_train import float_precision
    from experiments.Proceedings.n_train import sampler
elif experiment == "sampling_method":
    from experiments.Proceedings.sampling_method import n_train
    from experiments.Proceedings.sampling_method import float_precision
    from experiments.Proceedings.sampling_method import sampler
elif experiment == "float_precision":
    from experiments.Proceedings.float_precision import n_train
    from experiments.Proceedings.float_precision import float_precision
    from experiments.Proceedings.float_precision import sampler
else:
    raise Exception("Only n_train, sampling_method and float_precision experiments are specyfied.")

#### Exploring shape of approximated solutions

In [ ]:
def jupyter_plot_predictions(epsilon, error, n_train, float_precision, sampler):

    df, predictions, true_solutions = collect_data(data_experiment_name, k=k, alpha=alpha, sampler=[sampler], float_precision=[float_precision])
    fig, ax = plt.subplots(figsize=(12, 6))
    prediction_plot(df, predictions, true_solutions, ax, error, n_train, float_precision, sampler, epsilon)
    plt.show()
    
widgets.interact(
    jupyter_plot_predictions,
    epsilon=widgets.Dropdown(options=epsilons2try.tolist(), description=r'$\varepsilon$: '),
    error=widgets.Dropdown(options=[L2, H1], description='Error: '),
    n_train=widgets.Dropdown(options=n_train, description='N train: '),
    float_precision=widgets.Dropdown(options=float_precision, description='Float precision: '),
    sampler=widgets.Dropdown(options=sampler, description='Sampling method: ')
    )

#### Exploring error curves

In [5]:
summary_stattistic_functions_dict = {"min": np.min, "max": np.max, "median": np.median, "mean": np.mean} 

def jupyter_plot_error_curves(statistic, error, n_train, float_precision, sampler):
    aggfunc = summary_stattistic_functions_dict[statistic]
    df, predictions, true_solutions = collect_data(data_experiment_name, k=k, alpha=alpha, n_train=[n_train], sampler=[sampler], float_precision=[float_precision])
    fig, ax = plt.subplots(figsize=(12, 6))
    error_plots_flexible(df, ax, error_name=error, label_var="model_name", x_var="epsilon",
                         aggfunc=aggfunc, color_dict=models_color_dict, n_train=n_train)
    ax.set_ylim((1e-9, 1))
    plt.show()
    
widgets.interact(
    jupyter_plot_error_curves,
    statistic=widgets.Dropdown(options=list(summary_stattistic_functions_dict.keys()), description='Summary statistic: '),
    error=widgets.Dropdown(options=[L2, H1], description='Error: '),
    n_train=widgets.Dropdown(options=n_train, description='N train: '),
    float_precision=widgets.Dropdown(options=float_precision, description='Float precision: '),
    sampler=widgets.Dropdown(options=sampler, description='Sampling method: ')
    )

interactive(children=(Dropdown(description='Summary statistic: ', options=('min', 'max', 'median', 'mean'), va…

<function __main__.jupyter_plot_error_curves(statistic, error, n_train, float_precision, sampler)>

In [6]:
if experiment == "n_train":
    from experiments.Proceedings.n_train import n_train
    from experiments.Proceedings.n_train import float_precision
    from experiments.Proceedings.n_train import sampler
elif experiment == "sampling_method":
    from experiments.Proceedings.sampling_method import n_train
    from experiments.Proceedings.sampling_method import float_precision
    from experiments.Proceedings.sampling_method import sampler
elif experiment == "float_precision":
    from experiments.Proceedings.float_precision import n_train
    from experiments.Proceedings.float_precision import float_precision
    from experiments.Proceedings.float_precision import sampler
else:
    raise Exception("Only n_train, sampling_method and float_precision experiments are specyfied.")

#### Exploring shape of approximated solutions

In [7]:
def jupyter_plot_predictions(epsilon, error, n_train, float_precision, sampler):

    df, predictions, true_solutions = collect_data(data_experiment_name, k=k, alpha=alpha, sampler=[sampler], float_precision=[float_precision])
    fig, ax = plt.subplots(figsize=(12, 6))
    prediction_plot(df, predictions, true_solutions, ax, error, n_train, float_precision, sampler, epsilon)
    plt.show()
    
widgets.interact(
    jupyter_plot_predictions,
    epsilon=widgets.Dropdown(options=epsilons2try.tolist(), description=r'$\varepsilon$: '),
    error=widgets.Dropdown(options=[L2, H1], description='Error: '),
    n_train=widgets.Dropdown(options=n_train, description='N train: '),
    float_precision=widgets.Dropdown(options=float_precision, description='Float precision: '),
    sampler=widgets.Dropdown(options=sampler, description='Sampling method: ')
    )

interactive(children=(Dropdown(description='$\\varepsilon$: ', options=(0.005, 0.008, 0.014, 0.023, 0.039, 0.0…

<function __main__.jupyter_plot_predictions(epsilon, error, n_train, float_precision, sampler)>